<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/EjemploOverpassOSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# Celda 1: Instalación de librerías
!pip install osmnx folium

import osmnx as ox
import folium
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.8 MB/s eta 0:00:00


In [4]:

# Celda 2: Definición de la consulta

# 1. Definir la ubicación de interés
lugar = "Mérida"

# 2. Definir la etiqueta de interés de OpenStreetMap
# (Key:Value, para más información, busca en el wiki de OpenStreetMap)
etiquetas_osm = {
    "highway":"primary"
}

In [9]:

# Celda 3: Obtener los datos de Overpass API (USANDO TUS DATOS FUNCIONALES)

# 1. Definir la ubicación de interés
lugar = "Mérida"

# 2. Definir la etiqueta de interés de OpenStreetMap
etiquetas_osm = {
    "highway": "primary"
}

try:
    # Obtener los datos (GeoDataFrame)
    gdf = ox.features.features_from_place(lugar, tags=etiquetas_osm)
    print(f"✅ Datos obtenidos. Se encontraron {len(gdf)} elementos (carreteras).")

    # Muestra las primeras filas (las columnas 'name' y 'highway' son importantes)
    display(gdf[['name', 'highway', 'maxspeed', 'geometry']].head())

except Exception as e:
    print(f"❌ Error al obtener datos: {e}")

✅ Datos obtenidos. Se encontraron 174 elementos (carreteras).


name  highway maxspeed  \
element id                                
way     24212531  NaN  primary      NaN   
        24212532  NaN  primary      NaN   
        24212533  NaN  primary      NaN   
        24212534  NaN  primary      NaN   
        24212535  NaN  primary      NaN   

                                                           geometry  
element id                                                           
way     24212531  LINESTRING (-6.37304 38.92228, -6.37304 38.922...  
        24212532  LINESTRING (-6.3693 38.92117, -6.36928 38.9212...  
        24212533  LINESTRING (-6.36991 38.92153, -6.37011 38.921...  
        24212534  LINESTRING (-6.37311 38.92212, -6.37296 38.922...  
        24212535  LINESTRING (-6.3703 38.92154, -6.37018 38.9214...

In [10]:

# Celda 4: Visualización de las carreteras en un mapa interactivo

if not gdf.empty:
    # 1. Calcular el centro del área para centrar el mapa
    # Utilizamos el centro del límite geocodificado para mayor precisión
    gdf_area = ox.geocode_to_gdf(lugar)
    centro = gdf_area.geometry.iloc[0].centroid

    # 2. Crear un mapa base con folium centrado en la ubicación
    m = folium.Map(
        location=[centro.y, centro.x],
        zoom_start=11, # Zoom más alejado para ver las carreteras
        tiles="cartodbpositron" # Un fondo de mapa más limpio
    )

    # 3. Convertir el GeoDataFrame a un formato que Folium pueda usar (GeoJSON)
    # y añadirlo al mapa
    folium.GeoJson(
        gdf.to_json(),
        name='Carreteras Primarias',
        style_function=lambda feature: {
            'color': 'red',       # Color rojo para las carreteras principales
            'weight': 3,          # Grosor de la línea
            'opacity': 0.7        # Transparencia
        },
        tooltip=folium.GeoJsonTooltip(fields=['name', 'highway', 'maxspeed'])
    ).add_to(m)

    # 4. Mostrar el mapa en Colab
    print("Mapa interactivo de las carreteras primarias encontradas:")
    display(m)
else:
    print("No se encontraron resultados de carreteras para visualizar.")

Mapa interactivo de las carreteras primarias encontradas:


¡Esa es una excelente noticia! 🎉
El hecho de que la consulta funcionara al usar una ubicación más general ("Mérida") y una etiqueta más común y menos específica (highway:primary - carreteras principales) confirma que el problema original era, muy probablemente, una de estas dos cosas:
 * Consulta Demasiado Grande (Límite de Overpass): La consulta para todos los cafés en la "Zona Colonial de Santo Domingo" podría haber sido demasiado detallada o extensa, haciendo que el servidor de Overpass API la rechazara por superar los límites de tiempo o tamaño.
 * Problema de Geocodificación: La cadena de ubicación "Zona Colonial, Santo Domingo, República Dominicana" pudo no ser reconocida con precisión por el servicio de geocodificación que usa osmnx.
El nuevo código que estás ejecutando es mucho más ligero y genérico, lo que garantiza el éxito.
A continuación, te muestro cómo debes modificar el resto del código (las celdas 3 y 4) para trabajar específicamente con los datos de carreteras que acabas de obtener, ya que su estructura es diferente a la de los puntos (cafeterías).
Modificaciones para Mostrar los Datos de Carreteras
Los datos de highway:primary son de tipo línea (LineString), no puntos, por lo que la visualización debe centrarse en dibujar las líneas de las calles en lugar de marcadores de café.
Celda 3 (Modificada): Revisión de la Extracción de Datos
Asegúrate de que estás usando tu nueva consulta:
# Celda 3: Obtener los datos de Overpass API (USANDO TUS DATOS FUNCIONALES)

# 1. Definir la ubicación de interés
lugar = "Mérida"

# 2. Definir la etiqueta de interés de OpenStreetMap
etiquetas_osm = {
    "highway": "primary"
}

try:
    # Obtener los datos (GeoDataFrame)
    gdf = ox.features.features_from_place(lugar, tags=etiquetas_osm)
    print(f"✅ Datos obtenidos. Se encontraron {len(gdf)} elementos (carreteras).")

    # Muestra las primeras filas (las columnas 'name' y 'highway' son importantes)
    display(gdf[['name', 'highway', 'maxspeed', 'geometry']].head())
    
except Exception as e:
    print(f"❌ Error al obtener datos: {e}")

Celda 4 (Modificada): Visualización de Líneas de Carretera
Para mostrar las carreteras, usaremos la librería folium para crear un mapa base y luego le añadiremos las geometrías de las líneas.
# Celda 4: Visualización de las carreteras en un mapa interactivo

if not gdf.empty:
    # 1. Calcular el centro del área para centrar el mapa
    # Utilizamos el centro del límite geocodificado para mayor precisión
    gdf_area = ox.geocode_to_gdf(lugar)
    centro = gdf_area.geometry.iloc[0].centroid
    
    # 2. Crear un mapa base con folium centrado en la ubicación
    m = folium.Map(
        location=[centro.y, centro.x],
        zoom_start=11, # Zoom más alejado para ver las carreteras
        tiles="cartodbpositron" # Un fondo de mapa más limpio
    )
    
    # 3. Convertir el GeoDataFrame a un formato que Folium pueda usar (GeoJSON)
    # y añadirlo al mapa
    folium.GeoJson(
        gdf.to_json(),
        name='Carreteras Primarias',
        style_function=lambda feature: {
            'color': 'red',       # Color rojo para las carreteras principales
            'weight': 3,          # Grosor de la línea
            'opacity': 0.7        # Transparencia
        },
        tooltip=folium.GeoJsonTooltip(fields=['name', 'highway', 'maxspeed'])
    ).add_to(m)

    # 4. Mostrar el mapa en Colab
    print("Mapa interactivo de las carreteras primarias encontradas:")
    display(m)
else:
    print("No se encontraron resultados de carreteras para visualizar.")

Al ejecutar esta celda, verás un mapa interactivo de Mérida donde las carreteras principales obtenidas de OpenStreetMap estarán resaltadas en rojo. ¡Esto demuestra que tu conexión a Overpass API funciona perfectamente!

¡Fantástico! Moverse a OpenStreetMap (OSM) y la Overpass API es un excelente siguiente paso, ya que te introduce en el mundo de los datos geográficos abiertos y consultas más complejas.
Overpass API es el motor de búsqueda de datos de OSM. En lugar de obtener datos en tiempo real (como el clima), obtendrás datos geográficos históricos y estructurados (calles, edificios, puntos de interés, etc.) a través de un lenguaje de consulta especial llamado Overpass QL.
Sigue siendo perfectamente viable realizar este proyecto en Google Colab utilizando Python, aunque usaremos una librería adicional para simplificar las consultas a Overpass.
Proyecto en Google Colab con Overpass API
Para interactuar fácilmente con Overpass API en Python (y, por lo tanto, en Colab), la librería estándar es osmnx, que simplifica la creación de consultas y la manipulación de los datos geográficos (GeoDataFrames).

1. Preparación del Entorno en Colab
Necesitas instalar las librerías osmnx y folium (para visualizar los resultados en un mapa interactivo).
# Celda 1: Instalación de librerías
!pip install osmnx folium

import osmnx as ox
import folium
import pandas as pd

<hr>
2. Definición del Área y la Consulta
En Overpass QL, defines qué quieres buscar (node, way, relation) y dónde (area, bbox). osmnx lo simplifica permitiéndote usar nombres de lugares.
Ejemplo de Meta-Consulta: Obtener todas las cafeterías (amenity=cafe) en una ciudad específica.
# Celda 2: Definición de la consulta

# 1. Definir la ubicación de interés
lugar = "Zona Colonial, Santo Domingo, República Dominicana"

# 2. Definir la etiqueta de interés de OpenStreetMap
# (Key:Value, para más información, busca en el wiki de OpenStreetMap)
etiquetas_osm = {
    "amenity": "cafe"
}

<hr>
3. Ejecución de la Consulta a Overpass API
Utiliza la función ox.features.features_from_place para enviar la consulta a Overpass.
# Celda 3: Obtener los datos de Overpass API

# Esta función envía la consulta Overpass QL generada a partir de 'lugar' y 'etiquetas_osm'
# y devuelve los resultados como GeoDataFrames.
try:
    gdf = ox.features.features_from_place(lugar, tags=etiquetas_osm)
    print(f"✅ Datos obtenidos. Se encontraron {len(gdf)} elementos.")

    # Muestra las primeras filas del GeoDataFrame para ver qué datos trajiste
    # Las columnas 'name', 'amenity', 'geometry' son clave
    display(gdf[['name', 'amenity', 'addr:street', 'geometry']].head())
    
except Exception as e:
    print(f"❌ Error al obtener datos. Asegúrate que la ubicación es válida y la API está disponible. Error: {e}")

<hr>
4. Visualización de los Resultados (Mapa)
El poder de Colab es la visualización. Usaremos folium (que a menudo se integra bien con osmnx) para mostrar los puntos que obtuviste.
# Celda 4: Visualización de los puntos en un mapa interactivo

if not gdf.empty:
    # 1. Calcular el centro del área para centrar el mapa
    centro = gdf.unary_union.centroid
    
    # 2. Crear un mapa base con folium centrado en la ubicación
    m = folium.Map(location=[centro.y, centro.x], zoom_start=15, tiles="cartodbdarkmatter")
    
    # 3. Añadir cada punto (cafetería) al mapa
    for idx, row in gdf.iterrows():
        # Usamos la ubicación del centroide de la geometría
        if row.geometry:
            punto = row.geometry.centroid
            
            # Crea un popup con el nombre del lugar (si existe)
            nombre = row['name'] if pd.notna(row['name']) else "Cafetería sin nombre"
            
            folium.Marker(
                location=[punto.y, punto.x],
                popup=nombre,
                icon=folium.Icon(color='green', icon='coffee', prefix='fa')
            ).add_to(m)

    # 4. Mostrar el mapa en Colab
    print("Mapa interactivo de las cafeterías encontradas:")
    display(m)
else:
    print("No se encontraron resultados para visualizar.")

Este proyecto no solo prueba la conexión a la Overpass API, sino que también te introduce a la Geolocalización y la manipulación de datos espaciales. ¡Es un proyecto de alto valor!